In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from nlp import LemmaTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import pickle
from os.path import exists

In [4]:
df_dev = pd.read_csv("dataset/development.csv", index_col="ids")
df_eval = pd.read_csv("dataset/evaluation.csv", index_col="ids")

In [5]:
df_dev["sentiment"].value_counts()

1    130157
0     94837
Name: sentiment, dtype: int64

### Rimozione colonne inutili
Il campo flag è uguale per tutti gli esempi, il valore di user è basso per tutti i sample relativamente alla grandezza del dataset.
Forse si potrebbe raggruppare le date (ad esempio per periodo dell'anno) e categorizzarle con one-hot vector.

In [3]:
df_dev.drop(columns=["date", "flag", "user"], inplace=True)
df_eval.drop(columns=["date", "flag", "user"], inplace=True)

### Gestione sample duplicati
Ci sono alcuni elementi duplicati (riga intera). Questi vengono risolti con drop_duplicates keep='first'.
Tuttavia, solo in df_dev, ci sono alcuni tweet che si ripetono due volte con due valori di sentiment diversi. Questi vengono tenuti

In [4]:
df_dev.drop_duplicates(keep='first' ,inplace=True)
df_eval.drop_duplicates(keep='first', inplace=True)

In [5]:
# rimozione tweet duplicati con sentiment diverso
c = df_dev.index.value_counts()
c = c[c > 1]
indexes_to_remove = c.index.values
df_dev.drop(labels=indexes_to_remove, inplace=True)

### Document data preprocessing
Visto che i documenti sono corti (post di social) verranno analizzati nella loro interezza (vedi slide 60 - Data Preprocessing)
Gli step da compiere sono:
- document splitting (non eseguito perché i documenti sono corti)
- tokenisation (i token sono parole, documenti corti)
- case normalization (minuscolo)
- stopword removal
- stemming (trasformazione parole nella loro forma base)

In [6]:
force_computation = False
if force_computation == True or not exists("./Xdata.bin"):
    lemmaTokenizer = LemmaTokenizer(correct=False)
    vectorizer = TfidfVectorizer(stop_words='english', tokenizer=lemmaTokenizer)
    vectorizer.fit(df_dev["text"].values)
    Xdata = vectorizer.transform(df_dev["text"].values)
else:
    with open("./Xdata.bin", 'rb') as f:
        Xdata = pickle.load(f)
Ydata = df_dev["sentiment"].values

In [12]:
with open('Xdata.bin', 'wb') as f:
    pickle.dump(Xdata, f)

In [7]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xdata, Ydata, test_size=0.15)

In [11]:
model = LogisticRegression(solver='saga')
model.fit(Xtrain, Ytrain)
Ypred = model.predict(Xtest)
Ypred = np.round(Ypred, decimals=0).astype(int)
score = f1_score(Ytest, Ypred)
print(score)
# score: 81.5 %

0.8152969494522768


In [9]:
model = LinearSVC()
model.fit(Xtrain, Ytrain)
Ypred = model.predict(Xtest)
score = f1_score(Ytest, Ypred)
print(score)
# score: 80.6 %

0.8041851267803058


In [9]:
model = RandomForestClassifier(n_estimators=100, max_depth=20, max_features='sqrt')
model.fit(Xtrain, Ytrain)
Ypred = model.predict(Xtest)
score = f1_score(Ytest, Ypred)
print(score)

0.7328954362518035
